In [3]:
import pickle
import pandas as pd
import numpy as np
import os
from collections import defaultdict
import matplotlib.pyplot as plt


In [11]:
with open("WESAD/S3/S3_n0.pkl", 'rb') as file:
    data = pickle.load(file, encoding='bytes')
for device in ['wrist', 'chest']:
    for sensor in data[device]:
        print(f'{device}_{sensor}: {data[device][sensor].shape}')


wrist_ACC: (2930, 3, 32)
wrist_BVP: (2930, 1, 64)
wrist_EDA: (2930, 1, 4)
wrist_TEMP: (2930, 1, 4)
chest_ACC: (2930, 3, 700)
chest_ECG: (2930, 1, 700)
chest_EMG: (2930, 1, 700)
chest_EDA: (2930, 1, 700)
chest_Temp: (2930, 1, 700)
chest_Resp: (2930, 1, 700)


In [1]:
# Creating dataframe of all of them
# `window` | `mean` | `std` | `max` | `min` | `label`

from feature import extract_features

extract_features(verbose=1)


Data of S2 loaded!
Data of S3 loaded!
Data of S4 loaded!
Data of S5 loaded!
Data of S6 loaded!
Data of S7 loaded!
Data of S8 loaded!
Data of S9 loaded!
Data of S10 loaded!
Data of S11 loaded!
Data of S13 loaded!
Data of S14 loaded!
Data of S15 loaded!
Data of S16 loaded!
Data of S17 loaded!


In [1]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from training import WESADTrainer
import pandas as pd

subjects = {i for i in range(2, 18)}.difference({12})
trainer = WESADTrainer({
    RandomForestClassifier: 
        {'n_estimators': [100], 'min_samples_leaf': [1, 5]},
    KNeighborsClassifier:
        {'n_neighbors': [3, 7]},
    }, sensor='wrist_BVP', subjects=subjects, data_prefix="svd", classes=[2])

reports = trainer.predict_all(verbose=3)


Testing S2

Training RandomForestClassifier() completed in 82.78372931480408 secs
Training KNeighborsClassifier() completed in 8.959145307540894 secs
predicting df of size (2889, 5), made (2889,)
Report of S2: {'2': {'precision': 0.9927536231884058, 'recall': 0.8910569105691057, 'f1-score': 0.9391602399314483, 'support': 615}, '3': {'precision': 0.9713307659392384, 'recall': 0.9982409850483729, 'f1-score': 0.9846020386033398, 'support': 2274}, 'accuracy': 0.9754240221529941, 'macro avg': {'precision': 0.982042194563822, 'recall': 0.9446489478087393, 'f1-score': 0.961881139267394, 'support': 2889}, 'weighted avg': {'precision': 0.9758911872643466, 'recall': 0.9754240221529941, 'f1-score': 0.9749285508279111, 'support': 2889}}
Testing S3

Training RandomForestClassifier() completed in 86.80865597724915 secs
Training KNeighborsClassifier() completed in 16.57343888282776 secs
predicting df of size (2935, 5), made (2935,)
Report of S3: {'2': {'precision': 1.0, 'recall': 0.9390625, 'f1-score

In [3]:
for item in reports:
    print(f"{item}: {reports[item]}")

1: {'precision': 0, 'recall': 0, 'f1-score': 0}
2: {'precision': 0.9839628057282592, 'recall': 0.9712458971736574, 'f1-score': 0.9770378294281853}
3: {'precision': 0.9921134599943656, 'recall': 0.9953898923416481, 'f1-score': 0.9937115147355305}
4: {'precision': 0, 'recall': 0, 'f1-score': 0}
weighted avg: {'precision': 0.99030417042754, 'recall': 0.9901324086199886, 'f1-score': 0.9900643778326899}
macro avg: {'precision': 0.9880381328613124, 'recall': 0.9833178947576526, 'f1-score': 0.985374672081858}
accuracy: 0.9901324086199886


In [1]:
import torch
import numpy as np
from neuralNet import NeuralNet1, Trainer


window_size = 30 * 64
num_classes = 2
# Dummy input tensor: batch_size x sequence_length x input_size
# Dummy target tensor: batch_size
dummy_input = torch.from_numpy(np.random.rand(100, window_size).astype(np.float32))
dummy_target = torch.randint(0, num_classes, (100,)).type(torch.float32)

with open("WESAD/S3/S3_n0.pkl", 'rb') as file:
    data = pickle.load(file, encoding='bytes')


trainer = Trainer(data['wrist']['BVP'], data['labels'], window_size)
trainer.go()


Epoch [1/10], Loss: 0.7042
Epoch [2/10], Loss: 0.6921
Epoch [3/10], Loss: 0.6927
Epoch [4/10], Loss: 0.6929
Epoch [5/10], Loss: 0.6930
Epoch [6/10], Loss: 0.6930
Epoch [7/10], Loss: 0.6930
Epoch [8/10], Loss: 0.6928
Epoch [9/10], Loss: 0.6920
Epoch [10/10], Loss: 0.6855


In [12]:
import pandas as pd
df = pd.read_csv("WESAD/S3/svd_data/chest_ACC.csv")[::400]
display(df)
last_label = max([2]) + 1
for label in df['label'].unique():
    if label not in [2]:
        df.loc[df['label'] == label, 'label'] = last_label
display(df)
        

,0,1,2,3,window,label
0,-12.317579,-12.453173,-12.235504,-11.898505,354,1
400,-12.258131,-12.245710,-12.247555,-12.221802,754,1
800,-12.329714,-12.326698,-12.316850,-12.330651,1154,1
1200,-11.986158,-12.008317,-11.959359,-11.956366,2305,2
1600,-12.043670,-12.052217,-12.062527,-12.050158,2705,2
2000,-12.140834,-12.128213,-12.135657,-12.136818,4518,4
2400,-12.113762,-12.098755,-12.115757,-12.109707,5223,3
2800,-12.236691,-12.229245,-12.242570,-12.238531,6062,4


,0,1,2,3,window,label
0,-12.317579,-12.453173,-12.235504,-11.898505,354,3
400,-12.258131,-12.245710,-12.247555,-12.221802,754,3
800,-12.329714,-12.326698,-12.316850,-12.330651,1154,3
1200,-11.986158,-12.008317,-11.959359,-11.956366,2305,2
1600,-12.043670,-12.052217,-12.062527,-12.050158,2705,2
2000,-12.140834,-12.128213,-12.135657,-12.136818,4518,3
2400,-12.113762,-12.098755,-12.115757,-12.109707,5223,3
2800,-12.236691,-12.229245,-12.242570,-12.238531,6062,3
